In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding,Flatten,Dense

In [2]:
df = pd.read_csv('train.txt', sep = ';')
df.sample(3)

,i didnt feel humiliated,sadness
1682,i sat on a windy beach feeling thoroughly anno...,anger
725,im starting to not buy the whole everything ha...,anger
13250,i wont discuss any further made me feel really...,fear


In [3]:
df.columns = ['Text','Emotions']

In [4]:
df.columns

Index(['Text', 'Emotions'], dtype='object')

In [5]:
df.sample(5)

,Text,Emotions
11643,i was feeling discouraged at this point,sadness
5071,i am going crazy at leas the feeling is more p...,fear
1969,i write when i am feeling happy and childish,joy
8194,i will give proper praise to the amish for bei...,anger
15209,i have been feeling very discouraged the last ...,sadness


In [6]:
texts = df['Text'].tolist()
labels = df['Emotions'].tolist()

In [7]:
# Tokenize the text data
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(texts)

In [8]:
sequence = tokenizer.texts_to_sequences(texts)

In [9]:
l = []
for i in range(len(sequence)):
    l.append(len(sequence[i]))

In [10]:
max_len = l[np.argmax(l)]
max_len

66

In [11]:
padded_sequence = pad_sequences(sequence, maxlen = max_len)
padded_sequence

array([[   0,    0,    0, ...,    4,   22, 1255],
       [   0,    0,    0, ...,    3,  495,  438],
       [   0,    0,    0, ...,   30,    6, 3497],
       ...,
       [   0,    0,    0, ...,    4,  102, 1332],
       [   0,    0,    0, ...,  340,    9,   43],
       [   0,    0,    0, ...,   26, 3586,   13]])

In [12]:
#Label encoding 
label_encoder = LabelEncoder()
label = label_encoder.fit_transform(labels)

In [13]:
one_hot = keras.utils.to_categorical(label)
one_hot

array([[0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [14]:
x_train,x_test,y_train,y_test = train_test_split(padded_sequence,one_hot, test_size = 0.2, random_state = 42)

In [15]:
#model creation
model = Sequential()
model.add(Embedding(input_dim = len(tokenizer.word_index) + 1,output_dim = 128))
model.add(Flatten())
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = len(one_hot[0]), activation = 'softmax'))

model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
model.fit(x_train,y_train,epochs = 10, batch_size = 32 , validation_data = (x_test,y_test))

Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 16s 37ms/step - accuracy: 0.3928 - loss: 1.5140 - val_accuracy: 0.7056 - val_loss: 0.8349
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 15s 37ms/step - accuracy: 0.8888 - loss: 0.3812 - val_accuracy: 0.8197 - val_loss: 0.5495
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 15s 39ms/step - accuracy: 0.9874 - loss: 0.0558 - val_accuracy: 0.8284 - val_loss: 0.5872
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 16s 40ms/step - accuracy: 0.9956 - loss: 0.0242 - val_accuracy: 0.8244 - val_loss: 0.6124
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 15s 37ms/step - accuracy: 0.9966 - loss: 0.0160 - val_accuracy: 0.8156 - val_loss: 0.6658
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 17s 42ms/step - accuracy: 0.9976 - loss: 0.0146 - val_accuracy: 0.8197 - val_loss: 0.6816
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 15s 37ms/step - accuracy: 0.9969 - loss: 0.0111 - val_accuracy: 0.8231 - val_loss: 0.6831
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 15s 37ms/step - accuracy: 0.9980 - loss: 0.0093 - 

In [18]:
input_text = "i never make her separate from me because i don t ever want her to feel like i m ashamed with her"

input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_len)
prediction = model.predict(padded_input_sequence)
prediction_label = label_encoder.inverse_transform([np.argmax(prediction)])
print(prediction_label[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
sadness
